In [1]:
import os, sys
# import mcvine modules
from instrument.geometry.pml import weave
from instrument.geometry import  operations

In [5]:
thisdir = os.path.abspath(os.path.dirname("__file__"))
libpath = os.path.join(thisdir, '../c3dp_source')
if not libpath in sys.path:
    sys.path.insert(0, libpath)

In [6]:
import collimator_support as colli_original

In [7]:
import SCADGen.Parser
from create_collimator_geometry import Collimator_geom
from c3dp.instrument.cellgeometry.clampcell_geo import Clampcell
from c3dp.instrument.cellgeometry.DAC_geo import DAC

In [8]:
clampcell=Clampcell(total_height=True)
outer_body=clampcell.outer_body()
inner_sleeve=clampcell.inner_sleeve()
sample=clampcell.sample()

cell_sample_assembly=operations.unite(operations.unite(outer_body, sample), inner_sleeve)

In [9]:
dac=DAC()
anvil=dac.anvil()

# gasket=dac.gasket()
# gasket_holder=dac.gasket_holder()
sorrounding_gasket=dac.sorrounding_gasket()

seat=dac.seat()
piston=dac.piston()
seat_pistion=dac.seat_piston()
bar=dac.body_bar()
bar=dac.body_bar_rotated()

dac_cell=operations.unite(operations.unite(operations.unite(anvil,sorrounding_gasket),
                    seat_pistion), bar)
# dac_cell_beam= operations.rotate (dac_cell, transversal=1, angle='%s *degree' %(90))

In [10]:
collimator_fr_center=dac.gasket_diameter/2.
print (collimator_fr_center)

3.0


In [8]:
scad_flag = True  ########CHANGE CAD FLAG HERE

if scad_flag is True:
    samplepath = os.path.join(thisdir, '../figures')
else:
    samplepath = os.path.join(thisdir, '../sample')

In [9]:
colli_height=dac.pavilion_total_triangle_height()+dac.girdle_height+dac.seat_skirt_height+dac.seat_shaft_height+dac.piston_shaft_height+20
collimator_height=2*colli_height

print (collimator_height)

119.56781044332605


In [12]:
for coll_length in [collimator_height]: #100, 230,380


    channel_length=coll_length-15. #32, 17

    min_channel_wall_thickness=1.

    coll = colli_original.Collimator_geom()
#     coll = Collimator_geom()
    coll.set_constraints(max_coll_height_detector=collimator_height, max_coll_width_detector=collimator_height,
                        min_channel_wall_thickness=min_channel_wall_thickness,
                        max_coll_length=coll_length, min_channel_size=3.,
                         truss_base_thickness=10., trass_final_height_factor=0.34,
                         truss_blade_length=10,touch_to_halfcircle=3,
                         SNAP_acceptance_angle=False,
                        collimator_front_end_from_center=collimator_fr_center)


    coll.set_parameters(number_channels=5.,channel_length =channel_length)

    filename = 'coll_geometry_{coll_length}_{coll_height}_{coll_width}_{channel_length}_{wall_thickness}.xml'.\
        format(coll_length=coll_length, coll_height=coll.max_coll_height_detector, coll_width=coll.max_coll_height_detector, channel_length=channel_length, wall_thickness=min_channel_wall_thickness)

    outputfile = os.path.join(samplepath, filename)


    supports=coll.support()

    truss=coll.support_design()

    coli = coll.gen_one_col(collimator_Nosupport=True)
    
    coli_transversal = coll.gen_collimators(detector_angles=[90], 
                            multiple_collimator=False,collimator_Nosupport=False)

#     collimator=coll.gen_collimators_xml(multiple_collimator=False, scad_flag=scad_flag,detector_angles=[0], collimator_Nosupport=False, coll_file=outputfile)

In [12]:
# both=operations.unite(dac_cell_beam, coli_transversal)
# both= truss
# both=supports
# both=operations.unite(anvil, sorrounding_gasket)
both=operations.unite(dac_cell, coli)
file='dac_cell_coli'
filename='%s.xml'%(file)
outputfile=os.path.join(samplepath, filename)
with open (outputfile,'wt') as file_h:
    weave(both,file_h, print_docs = False)

In [13]:
p = SCADGen.Parser.Parser(outputfile)
p.createSCAD()
test = p.rootelems[0]

In [14]:
cadFile_name='%s.scad'%(file)
cad_file_path=os.path.abspath(os.path.join(samplepath, cadFile_name))

In [15]:
cad_file_path

'/home/fi0/Collimator_3D/c3dp/figures/dac_cell_coli.scad'

In [ ]:
!vglrun openscad {cad_file_path}